# Pruning a CNN for CIFAR-10 dataset

## Import

In [2]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as date

## Prepare data

In [3]:
# obtain dataset and display size
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
(x_train.shape, x_test.shape)

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [4]:
# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [5]:
# normalize training data
x_train_R = x_train.transpose()[0].transpose()
x_train_G = x_train.transpose()[1].transpose()
x_train_B = x_train.transpose()[2].transpose()

# norm_vals = np.array([[0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]])

def norm(arr):
    return (arr - np.mean(arr)) / np.std(arr)

x_train_R = norm(x_train_R)
x_train_G = norm(x_train_G)
x_train_B = norm(x_train_B)

x_train = np.array([x_train_R, x_train_G, x_train_B])
x_train = np.moveaxis(x_train, 0, -1)

x_train.shape

(50000, 32, 32, 3)

In [6]:
# normalize test data
x_test_R = x_test.transpose()[0].transpose()
x_test_G = x_test.transpose()[1].transpose()
x_test_B = x_test.transpose()[2].transpose()

x_test_R = norm(x_test_R)
x_test_G = norm(x_test_G)
x_test_B = norm(x_test_B)

x_test = np.array([x_test_R, x_test_G, x_test_B])
x_test = np.moveaxis(x_test, 0, -1)

x_test.shape

(10000, 32, 32, 3)

In [7]:
# verify normalization
print('Mean:', np.mean(x_train))
print('StDev:', np.std(x_train))

print('Mean:', np.mean(x_test))
print('StDev:', np.std(x_test))

Mean: 5.4830214442821065e-17
StDev: 1.0000000000000002
Mean: 8.14903700074865e-18
StDev: 1.0000000000000002


## Convolutional Neural Network

In [21]:
# build architecture, compile, display summary
cnn = tf.keras.models.Sequential([
    # preprocessing
    tf.keras.layers.InputLayer(input_shape=(32, 32, 3)),
    # tf.keras.layers.experimental.preprocessing.RandomFlip(),
    # tf.keras.layers.experimental.preprocessing.RandomCrop(28, 28),
    # convolution and pooling
    tf.keras.layers.Conv2D(filters=32, activation='relu', kernel_size=(3,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(filters=64, activation='relu', kernel_size=(3,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    # dense
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               691500    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

## Train with callbacks

In [12]:
# setup callbacks
log_dir = "C:/Users/andre/neural/log/" + date.now().strftime("%Y-%m-%d/") + date.now().strftime("%H-%M-%S")
print(log_dir)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq=1)

# train
training = cnn.fit(x_train, y_train, epochs=20, callbacks=[tb_callback])

C:/Users/andre/neural/log/2021-06-23/11-44-49
Epoch 1/20
1563/1563 [==============================] - 13s 7ms/step - loss: 1.5102 - accuracy: 0.4564
Epoch 2/20
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9029 - accuracy: 0.6815
Epoch 3/20
1563/1563 [==============================] - 10s 7ms/step - loss: 0.7066 - accuracy: 0.7531
Epoch 4/20
1563/1563 [==============================] - 10s 7ms/step - loss: 0.5430 - accuracy: 0.8137
Epoch 5/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.3980 - accuracy: 0.8620
Epoch 6/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.2770 - accuracy: 0.9030
Epoch 7/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.1864 - accuracy: 0.9362
Epoch 8/20
1563/1563 [==============================] - 10s 7ms/step - loss: 0.1404 - accuracy: 0.9537
Epoch 9/20
1563/1563 [==============================] - 10s 7ms/step - loss: 0.1050 - accuracy: 0.9641
Epoch 10/20
1563/1563 [====

In [16]:
# evaluate
_, test_accuracy = cnn.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 2.4617 - accuracy: 0.7124


## Pruning every layer

In [32]:
# create model for pruning (all layers will be pruned)
cnn_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(cnn)

# find a way to print out the sparsity schedule

# check to make sure layers are set for pruning
cnn_for_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_2 (None, 30, 30, 32)        1762      
_________________________________________________________________
prune_low_magnitude_max_pool (None, 15, 15, 32)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_3 (None, 13, 13, 64)        36930     
_________________________________________________________________
prune_low_magnitude_max_pool (None, 6, 6, 64)          1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 2304)              1         
_________________________________________________________________
prune_low_magnitude_dense_2  (None, 300)               1382702   
_________________________________________________________________
prune_low_magnitude_dense_3  (None, 10)               

C:\Users\andre\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


## Train CNN with pruning

In [33]:
# log directory
log_dir = "C:/Users/andre/neural/log/" + date.now().strftime("%Y-%m-%d/") + date.now().strftime("%H-%M-%S")
print(log_dir)

# special callback to log pruning data
callbacks = [
  #tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq=1),
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir, update_freq=1)
]

cnn_for_pruning.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

pruned_training = cnn_for_pruning.fit(x_train, y_train, callbacks=callbacks, epochs=5)

C:/Users/andre/neural/log/2021-06-23/17-02-04
Epoch 1/5
1563/1563 [==============================] - 36s 22ms/step - loss: 0.6482 - accuracy: 0.7731
Epoch 2/5
1563/1563 [==============================] - 49s 31ms/step - loss: 0.5139 - accuracy: 0.8242
Epoch 3/5
1563/1563 [==============================] - 61s 39ms/step - loss: 0.4242 - accuracy: 0.8551
Epoch 4/5
1563/1563 [==============================] - 54s 35ms/step - loss: 0.3382 - accuracy: 0.8837
Epoch 5/5
1563/1563 [==============================] - 38s 25ms/step - loss: 0.2624 - accuracy: 0.9141


In [10]:
# evaluate
_, test_accuracy = cnn.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 2.3411 - accuracy: 0.7007


## Pruning only some layers

In [12]:
# designate only dense layers to be pruned
def prune_dense(layer):
    if isinstance(layer, tf.keras.layers.Dense):
        return tfmot.sparsity.keras.prune_low_magnitude(layer)
    return layer

# clone the model and apply this function to each layer
cnn_for_dense_pruning = tf.keras.models.clone_model(cnn, clone_function=prune_dense)

# check to make sure only dense layers are affected
cnn_for_dense_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
prune_low_magnitude_dense_2  (None, 300)               1382702   
_________________________________________________________________
prune_low_magnitude_dense_3  (None, 10)               

C:\Users\andre\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


## Train CNN with dense pruning

In [13]:
# log directory
log_dir = "C:/Users/andre/neural/log/" + date.now().strftime("%Y-%m-%d/") + date.now().strftime("%H-%M-%S")
print(log_dir)

# special callback to log pruning data
callbacks = [
  #tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq=1),
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir, update_freq=1)
]

cnn_for_dense_pruning.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

dense_pruned_training = cnn_for_dense_pruning.fit(x_train, y_train, callbacks=callbacks, epochs=20)

C:/Users/andre/neural/log/2021-06-23/14-36-45
Epoch 1/20
1563/1563 [==============================] - 17s 10ms/step - loss: 1.4967 - accuracy: 0.4657
Epoch 2/20
1563/1563 [==============================] - 15s 10ms/step - loss: 0.9273 - accuracy: 0.6799
Epoch 3/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.7575 - accuracy: 0.7350
Epoch 4/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.6105 - accuracy: 0.7886
Epoch 5/20
1563/1563 [==============================] - 15s 9ms/step - loss: 0.5017 - accuracy: 0.8286
Epoch 6/20
1563/1563 [==============================] - 16s 10ms/step - loss: 0.3920 - accuracy: 0.8632
Epoch 7/20
1563/1563 [==============================] - 15s 10ms/step - loss: 0.3041 - accuracy: 0.8963
Epoch 8/20
1563/1563 [==============================] - 18s 11ms/step - loss: 0.2270 - accuracy: 0.9230
Epoch 9/20
1563/1563 [==============================] - 16s 10ms/step - loss: 0.1703 - accuracy: 0.9409
Epoch 10/20
1563/1563

In [14]:
# evaluate
_, test_accuracy = cnn.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 2.3015 - accuracy: 0.7020


## Compare sizes

In [34]:
# we are mainly interested in the size of weights files
cnn.save_weights("cnn_weights.h5")
os.path.getsize("cnn_weights.h5")

2875408

In [35]:
cnn_for_pruning.save_weights("cnn_for_pruning_weights.h5")
os.path.getsize("cnn_for_pruning_weights.h5")

5742908

In [39]:
cnn.save("cnn_weights")
os.path.getsize("cnn_weights")

INFO:tensorflow:Assets written to: cnn_weights\assets


4096

In [37]:
cnn_for_pruning.save("cnn_for_pruning_weights.h5")
os.path.getsize("cnn_for_pruning_weights.h5")

11480268

In [38]:
import tempfile

def get_gzipped_model_size(model):
    import os
    import zipfile

    _, keras_file = tempfile.mkstemp('.h5')
    
    # print(os.path.getsize(keras_file))
    model.save(keras_file, include_optimizer=False)

    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    
    return os.path.getsize(zipped_file)

# get_gzipped_model_size(cnn) # 2.6 MB

# strip_pruning is necessary to see the compression benefits of pruning
z_cnn = tfmot.sparsity.keras.strip_pruning(cnn)

#get_gzipped_model_size(z_cnn)

z_cnn_for_pruning = tfmot.sparsity.keras.strip_pruning(cnn_for_pruning)


# z_cnn_for_dense_pruning = tfmot.sparsity.keras.strip_pruning(cnn_for_dense_pruning)
print("Size of gzipped model: %.2f bytes" % (get_gzipped_model_size(cnn)))
print("Size of gzipped pruned model: %.2f bytes" % (get_gzipped_model_size(cnn_for_pruning)))

print("Size of gzipped model: %.2f bytes" % (get_gzipped_model_size(z_cnn)))
print("Size of gzipped pruned model: %.2f bytes" % (get_gzipped_model_size(z_cnn_for_pruning)))
# print("Size of gzipped dense pruned model: %.2f bytes" % (get_gzipped_model_size(cnn_for_dense_pruning)))

Size of gzipped model: 1656946.00 bytes
Size of gzipped pruned model: 1838780.00 bytes
Size of gzipped model: 1656946.00 bytes
Size of gzipped pruned model: 1656946.00 bytes


# Next

experiment with the effect that sparsity has (for loop of 0.1) on the accuracy of pruned networks
quantization
research how the models are being compressed
research save_weights()